# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 70 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.04452


extracting tarball to tmp_2209.04452...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.04453


extracting tarball to tmp_2209.04453...

...

extracting tarball to tmp_2209.05242...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.05303


extracting tarball to tmp_2209.05303...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.05303/arxiv.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04914-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04914) | **AGN feedback in NGC 3982**  |
|| <mark>Prajwel Joseph</mark>, Koshy George, K. T. Paul |
|*Appeared on*| *2022-09-13*|
|*Comments*| *Accepted in A&A, 7 pages, 8 figures, interactive online figure at this https URL*|
|**Abstract**| The energetic feedback from supermassive black holes can influence star formation at the centres of galaxies. Observational evidence for AGN impact on star formation can be searched in galaxies by combining ultraviolet imaging and optical integral field unit data. The ultraviolet flux directly traces recent star formation, and the integral field unit data can reveal dust attenuation, gas ionisation mechanisms, and gas/stellar kinematics from the central regions of the galaxy disk. A pilot study on NGC 3982 shows star formation suppression in the central regions of the galaxy, likely due to negative AGN feedback, and enhanced star formation in the outer regions. The case of NGC 3982 could be observational evidence of AGN feedback operating in a Seyfert galaxy. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05125-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05125) | **X-ray detection of a nova in the fireball phase**  |
|| <mark>Ole König</mark>, et al. -- incl., <mark>Frank Haberl</mark> |
|*Appeared on*| *2022-09-13*|
|*Comments*| *Published in Nature on 11 May 2022*|
|**Abstract**| Novae are caused by runaway thermonuclear burning in the hydrogen-rich envelopes of accreting white dwarfs, which results in the envelope to expand rapidly and to eject most of its mass. For more than 30 years, nova theory has predicted the existence of a "fireball" phase following directly the runaway fusion, which should be observable as a short, bright, and soft X-ray flash before the nova becomes visible in the optical. Here we present the unequivocal detection of an extremely bright and very soft X-ray flash of the classical Galactic nova YZ Reticuli 11 hours prior to its 9 mag optical brightening. No X-ray source was detected 4 hours before and after the event, constraining the duration of the flash to shorter than 8 hours. In agreement with theoretical predictions, the source's spectral shape is consistent with a black body of $3.27^{+0.11}_{-0.33}\times 10^5$ K ($28.2^{+0.9}_{-2.8}$ eV), or a white dwarf atmosphere, radiating at the Eddington luminosity, with a photosphere that is only slightly larger than a typical white dwarf. This detection of the expanding white dwarf photosphere before the ejection of the envelope provides the last link of the predicted photospheric lightcurve evolution and opens a new window to measure the total nova energetics. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04452-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04452) | **A characterization method for low-frequency environmental noise in LIGO**  |
|| Guillermo Valdes, et al. -- incl., <mark>Yanqi Zhang</mark> |
|*Appeared on*| *2022-09-13*|
|*Comments*| **|
|**Abstract**| We present a method to characterize the noise in ground-based gravitational-wave observatories such as the Laser Gravitational-Wave Observatory (LIGO). This method uses linear regression algorithms such as the least absolute shrinkage and selection operator (LASSO) to identify noise sources and analyzes the detector output versus noise witness sensors to quantify the coupling of such noise. Our method can be implemented with currently available resources at LIGO, which avoids extra coding or direct experimentation at the LIGO sites. We present two examples to validate and estimate the coupling of elevated ground motion at frequencies below 10 Hz with noise in the detector output. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05205-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05205) | **The KOBE experiment: K-dwarfs Orbited By habitable Exoplanets. Project  goals, target selection and stellar characterization**  |
|| J. Lillo-Box, et al. -- incl., <mark>I. Hermelo</mark> |
|*Appeared on*| *2022-09-13*|
|*Comments*| *Accepted for publication in A&A. 12 pages, 9 figures, 1 table*|
|**Abstract**| The detection of habitable worlds is one of humanity's greatest endeavors. So far, astrobiological studies show that one of the most critical components for life development is liquid water. Its chemical properties and its capacity to dissolve and hence transport other substances makes this constituent a key piece in the development of life. As a consequence, looking for life as we know it is directly related to the search for liquid water. For a remote detection of life in distant planetary systems, this means looking for planets in the so-called habitable zone. In this sense, K-dwarf stars are the perfect hosts. Contrary to G-dwarfs, the habitable zone is closer, thus making planet detection easier using transit or radial velocity techniques. Contrary to M-dwarfs, the stellar activity is much smaller, hence having a smaller impact in both the detectability and in the true habitability of the planet. Also, K-dwarfs are the quietest in terms of oscillations, and granulation noise. Despite this, there is a dearth of planets in the habitable zone of K-dwarfs due to a lack of observing programs devoted to this parameter space. In response to a call for Legacy Programs of the Calar Alto observatory, we have started the first dedicated and systematic search for habitable planets around K-dwarfs, the K-dwarfs Orbited By habitable Exoplanets (KOBE). This survey is monitoring the radial velocity of 50 carefully pre-selected K-dwarfs with the CARMENES instrument along 5 semesters with an average of 90 data points per target. Based on planet occurrence rates convolved with our detectability limits, we expect to find $1.68\pm 0.25$ planets per star in the KOBE sample and in half of the sample we expect to find one of those planets within the habitable zone. In this paper, we describe the project motivation, goals and target selection and preliminary stellar characterization. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05229-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05229) | **Resilience of small PAHs in interstellar clouds: Efficient stabilization  of cyanonaphthalene by fast radiative cooling**  |
|| Mark H. Stockett, et al. -- incl., <mark>Henrik Cederquist</mark>, <mark>Henning T. Schmidt</mark>, <mark>Henning Zettergren</mark> |
|*Appeared on*| *2022-09-13*|
|*Comments*| **|
|**Abstract**| After decades of speculation and searching, astronomers have recently identified specific Polycyclic Aromatic Hydrocarbons (PAHs) in space. Remarkably, the observed abundance of cyanonaphthalene (CNN, C10H7CN) in the Taurus Molecular Cloud (TMC-1) is six orders of magnitude higher than expected from astrophysical modeling. Here, we report absolute unimolecular dissociation and radiative cooling rate coefficients of the 1-CNN isomer in its cationic form. These results are based on measurements of the time-dependent neutral product emission rate and Kinetic Energy Release distributions produced from an ensemble of internally excited 1-CNN + studied in an environment similar to that in interstellar clouds. We find that Recurrent Fluorescence - radiative relaxation via thermally populated electronic excited states - efficiently stabilizes 1-CNN+ , owing to a large enhancement of the electronic transition probability by vibronic coupling. Our results help explain the anomalous abundance of CNN in TMC-1 and challenge the widely accepted picture of rapid destruction of small PAHs in space. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05242-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05242) | **Rotation Measure Variations and Reversals of Repeating Fast Radio Bursts  in Massive Binary Systems**  |
|| Z. Y. Zhao, <mark>G. Q. Zhang</mark>, F. Y. Wang, Z. G. Dai |
|*Appeared on*| *2022-09-13*|
|*Comments*| *16 pages, 4 figures, 2 tables, submitted*|
|**Abstract**| Recent observations discovered that some repeating fast radio bursts (FRBs) show complex variations and reversals of Faraday rotation measures (RMs), indicating the sources of these FRBs are embedded in a dynamically magnetized environment. One possible scenario is that repeating FRBs generated by pulsars in binary systems, especially containing a high-mass companion with strong stellar outflows. Here, we study the RM variations caused by stellar winds, and a possible stellar disc. If the magnetic field is radial in the stellar wind, RM will reach the peak at periastron and will not reverse. For the toroidal magnetic field in the wind, RM will reverse at the super conjunction. The $\left| \mathrm{RM} \right|$ evolution is symmetric before and after the periastron for a radial magnetic field, or the super conjunction for a toroidal magnetic field. For the case of the toroidal field in the disc, the RM variations only occur at the time when the pulsar passes through the inclined disc before and after periastron. Our model can explain the secular RM variation of FRB 20180916B. Besides, the clumps in the stellar wind and disc can cause short time-scale ($< 1$ day) variations or reversals of RM. Therefore, long-term monitoring of RM variations can reveal the origins of FRBs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.05303-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.05303) | **The phase curve and the geometric albedo of WASP-43b measured with  CHEOPS, TESS and HST WFC3/UVIS**  |
|| G. Scandariato, et al. -- incl., <mark>K. Heng</mark>, <mark>S. Hoyer</mark>, <mark>M. B. Davies</mark>, <mark>G. Olofsson</mark> |
|*Appeared on*| *2022-09-13*|
|*Comments*| **|
|**Abstract**| Observations of the phase curves and secondary eclipses of extrasolar planets provide a window on the composition and thermal structure of the planetary atmospheres. For example, the photometric observations of secondary eclipses lead to the measurement of the planetary geometric albedo $A_g$, which is an indicator of the presence of clouds in the atmosphere. In this work we aim to measure the $A_g$ in the optical domain of WASP-43b, a moderately irradiated giant planet with an equilibrium temperature of $\sim$1400~K. To this purpose, we analyze the secondary eclipse light curves collected by CHEOPS, together with TESS observations of the system and the publicly available photometry obtained with HST WFC3/UVIS. We also analyze the archival infrared observations of the eclipses and retrieve the thermal emission spectrum of the planet. By extrapolating the thermal spectrum to the optical bands, we correct the optical eclipses for thermal emission and derive the optical $A_g$. The fit of the optical data leads to a marginal detection of the phase curve signal, characterized by an amplitude of $160\pm60$~ppm and 80$^{+60}_{-50}$~ppm in the CHEOPS and TESS passband respectively, with an eastward phase shift of $\sim50^\circ$ (1.5$\sigma$ detection). The analysis of the infrared data suggests a non-inverted thermal profile and solar-like metallicity. The combination of optical and infrared analysis allows us to derive an upper limit for the optical albedo of $A_g<0.087$ with a confidence of 99.9\%. Our analysis of the atmosphere of WASP-43b places this planet in the sample of irradiated hot Jupiters, with monotonic temperature-pressure profile and no indication of condensation of reflective clouds on the planetary dayside. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.04607-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.04607) | **Brown-dwarf companions in microlensing binaries detected during the  2016--2018 seasons**  |
|| Cheongho Han, et al. -- incl., <mark>Andrew Gould</mark>, <mark>Seung-Lee Kim</mark>, <mark>Chung-Uk Lee</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark> |
|*Appeared on*| *2022-09-13*|
|*Comments*| *11 pages, 10 figures, 10 tables*|
|**Abstract**| With the aim of finding microlensing binaries containing brown-dwarf (BD) companions, we investigate the microlensing survey data collected during the 2016--2018 seasons. For this purpose, we first conducted modeling of lensing events with light curves exhibiting anomaly features that are likely to be produced by binary lenses. We then sorted out BD-companion binary-lens events by applying the criterion that the companion-to-primary mass ratio is $q \lesssim 0.1$. From this procedure, we identify 6 binaries with candidate BD companions, including OGLE-2016-BLG-0890L, MOA-2017-BLG-477L, OGLE-2017-BLG-0614L, KMT-2018-BLG-0357L, OGLE-2018-BLG-1489L, and OGLE-2018-BLG-0360L. We estimate the masses of the binary companions by conducting Bayesian analyses using the observables of the individual lensing events. According to the Bayesian estimation of the lens masses, the probabilities for the lens companions of the events OGLE-2016-BLG-0890, OGLE-2017-BLG-0614, OGLE-2018-BLG-1489, and OGLE-2018-BLG-0360 to be in the BD mass regime are very high with $P_{\rm BD}> 80\%$. For MOA-2017-BLG-477 and KMT-2018-BLG-0357, the probabilities are relatively low with $P_{\rm BD}=61\%$ and 69\%, respectively. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

30  publications in the last 7 days.
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers